# Rethinking Statistics course in pymc3 - Week 7

Lecture 13: Monsters & Mixtures (Poisson GLMs, survival, zero-inflation)

- [Video](https://www.youtube.com/watch?v=p7g-CgGCS34)
- [Slides](https://speakerdeck.com/rmcelreath/l13-statistical-rethinking-winter-2019)

Lecture 14: Ordered Categories, Left & Right

- [Video](https://www.youtube.com/watch?v=zA3Jxv8LOrA)
- [Slides](https://speakerdeck.com/rmcelreath/l14-statistical-rethinking-winter-2019)

[Proposed problems](https://github.com/gbosquechacon/statrethinking_winter2019/blob/master/homework/week07.pdf) and [solutions in R](https://github.com/gbosquechacon/statrethinking_winter2019/blob/master/homework/week07_solutions.pdf) for the exercises of the week.

In [1]:
%matplotlib inline
import pymc3 as pm
import numpy as np
import pandas as pd
import theano
import theano.tensor as tt
from scipy import stats
from scipy.special import expit as logistic
from sklearn import preprocessing

import matplotlib.pyplot as plt
import altair as alt
alt.data_transformers.enable('default', max_rows=None)
import arviz as az

import warnings
warnings.filterwarnings('ignore')

## Exercise 1

> In the Trolley data, `Trolley`, we saw how education level (modeled as an ordered category) is associated with responses. Is this association causal? One plausible confound is that education is also associated with age, through a causal process: People are older when they finish school than when they begin it.

> Reconsider the `Trolley` data in this light. Draw a DAG that represents hypothetical causal relationships among response, education, and age. Which statical model or models do you need to evaluate the causal influence of education on responses? Fit these models to the trolley data. What do you conclude about the causal relationships among these three variables?

This is my DAG:

<img src="../../images/w7_img1.png" width="30%">

Age could influence both response `R` and education `E`. It could influence `R`, because people at different ages could have different attitudes. Age could influence education, because the longer you have lived, the more education you could have completed (up to a point). It's like the age causing marriage example from earlier in the course.

To evaluate the causal influence of `E` on `R`, we need to block the back-door from `E` through `A` to `R`. So we need a model that conditions on both `E` and `A`. Then the estimate for `E` should be the causal influence of `E`.

_Let's get the data._

In [4]:
d = pd.read_csv('../../data/Trolley.csv', header=0, sep=';')

elvl = d['edu'].unique()
idx = [7 , 0 , 6 , 4 , 2 , 1, 3, 5]
cat = pd.Categorical(d.edu, categories=list(elvl[idx]), ordered=True)
d['edu_new'] = pd.factorize(cat, sort=True)[0].astype('Int64')

#d['edu_norm'] = preprocessing.normalize(np.array(d['edu_new']).reshape(-1, 1), axis = 0, norm='max')
d['age_std'] = preprocessing.scale(d['age']) # note the standarization here on age

d.head()

,case,response,order,id,age,male,edu,action,intention,contact,story,action2,edu_new,age_std
0,cfaqu,4,2,96;434,14,0,Middle School,0,0,1,aqu,1,1,-1.650358
1,cfbur,3,31,96;434,14,0,Middle School,0,0,1,bur,1,1,-1.650358
2,cfrub,4,16,96;434,14,0,Middle School,0,0,1,rub,1,1,-1.650358
3,cibox,3,32,96;434,14,0,Middle School,0,1,1,box,1,1,-1.650358
4,cibur,3,4,96;434,14,0,Middle School,0,1,1,bur,1,1,-1.650358


In [5]:
action = theano.shared(d['action'].values)
contact = theano.shared(d['contact'].values)
age_std = theano.shared(d['age_std'].values)
intention = theano.shared(d['intention'].values)
male = theano.shared(d['male'].values)
response = theano.shared(d['response'].values)

_In order to use the educational level as a categorical independent variable we need to reshape it as a matrix (defined in the below cell as_ `edu_new_total`).

_It's not a particularly elegant solution but I could not find a better workaround. Inspiration came directly from_ [this](https://github.com/junpenglao/Planet_Sakaar_Data_Science/blob/master/Ports/brms_monotonic_compare.ipynb) _notebook from one of the core members of `pymc3`. The syntax proposed in the original exercise in R (through `ulam` and `alist`) it's not great either._

In [6]:
a = theano.shared(np.zeros(1,))
edu_new = d['edu_new'].values
edu_new_total = np.zeros((len(edu_new), 8), int)
for ic, i in enumerate(edu_new):
    if i > 0:
        edu_new_total[ic, :i+1] = np.arange(i+1)

_For instance, individual number 50 (index 49) reached the sixth level of education (six levels from 0 to 5)._

In [18]:
edu_new_total[49]

array([0, 1, 2, 3, 4, 5, 0, 0])

_For the dependent variable `response` is much easier using the `OrderedLogistc` function in `pymc3` and changing a bit the structure of the Normal that defines `cutpoints`._

In [21]:
with pm.Model() as model_1:
    # Data is defined outside of the model
    
    # Priors
    cutpoints = pm.Normal('cutpoints',
                          mu=0,
                          sd=1.5,
                          transform=pm.distributions.transforms.ordered,
                          shape=len(d.response.unique())-1,
                          testval=np.arange(len(d.response.unique())-1) - 2.5)
    bA = pm.Normal('bA', mu=0, sd=0.5)
    bC = pm.Normal('bC', mu=0, sd=0.5)
    bAge = pm.Normal('bAge', mu=0, sd=0.5)
    bE = pm.Normal('bE', mu=0, sd=0.5)
    bI = pm.Normal('bI', mu=0, sd=0.5)
    bIA = pm.Normal('bIA', mu=0, sd=0.5)
    bIC = pm.Normal('bIC', mu=0, sd=0.5)
    edu_lvl_coefs = pm.Dirichlet('edu_lvl_coefs', a=np.ones(7)) # We need 7 coefficients for the levels of education because we have 8 levels
    edu_lvl_aux = tt.concatenate([a, edu_lvl_coefs]) # This is just and auxiliary variable for the 8 levels of education
    
    # Regression
    BI = bI + bIA*action + bIC*contact
    phi = bA*action + bC*contact + bAge*age_std + BI*intention + bE*tt.sum(edu_lvl_aux[edu_new_total], axis=1)
    response_hat = pm.OrderedLogistic('response_hat', phi, cutpoints, observed=response-1) # note the -1 here, because the response have to start at level 0

    # Prior sampling, trace definition and posterior sampling
    #prior = pm.sample_prior_predictive()
    posterior_1 = pm.sample(tune=1000)
    #posterior_pred_1 = pm.sample_posterior_predictive(posterior_1)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [edu_lvl_coefs, bIC, bIA, bI, bE, bAge, bC, bA, cutpoints]
Sampling 4 chains, 0 divergences: 100%|██████████| 6000/6000 [05:27<00:00,  8.37draws/s]


In [22]:
az.summary(posterior_1, credible_interval=.89).round(2)

,mean,sd,mcse_mean,mcse_sd,hpd_5.5%,hpd_94.5%,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
bA,-0.47,0.05,0.0,0.0,-0.56,-0.39,1095.0,1095.0,1097.0,1270.0,1.00
bC,-0.34,0.07,0.0,0.0,-0.44,-0.22,1067.0,1067.0,1068.0,1273.0,1.00
bAge,-0.11,0.02,0.0,0.0,-0.14,-0.08,2584.0,2584.0,2585.0,1311.0,1.00
bE,0.25,0.06,0.0,0.0,0.16,0.35,1219.0,1120.0,1284.0,1083.0,1.00
bI,-0.29,0.06,0.0,0.0,-0.38,-0.20,930.0,928.0,930.0,1202.0,1.00
bIA,-0.44,0.08,0.0,0.0,-0.57,-0.32,1100.0,1087.0,1103.0,1381.0,1.00
bIC,-1.25,0.10,0.0,0.0,-1.40,-1.09,1134.0,1132.0,1142.0,1440.0,1.00
cutpoints[0],-2.47,0.07,0.0,0.0,-2.58,-2.37,844.0,844.0,859.0,1050.0,1.00
cutpoints[1],-1.77,0.07,0.0,0.0,-1.87,-1.66,832.0,832.0,839.0,1060.0,1.00
cutpoints[2],-1.18,0.07,0.0,0.0,-1.28,-1.08,860.0,860.0,868.0,997.0,1.00


> You may recall from the chapter that education has a negative effect in the model without age. Now that we include age, education has a positive influence (with some overlap with zero). So age has indeed soaked up some of the previous influence assigned to education. The back-door may be real.

> I'd summarize this model, assuming this DAG is true, as saying that age causes people to give slightly lower responses. This could be a cohort effect, and not a causal influence of age. Either way, it is small. Education seems to cause higher responses (more approval). This suggests that education trains people to see some or all of the features A, I, C as more permissible. A model that interacted education with each might shed more light on things. Remember: A DAG doesn't say whether you need an interaction effect or not. That is a separate problem.

## Exercise 2

> Consider one more variable in the Trolley data: Gender. Suppose that gender might influence education as well as response directly. Draw the DAG now that includes response, education, age, and gender.

> Using only the DAG, is it possible that the inferences from Problem 1 are confounded by gender? If so, define any additional models you need to infer the causal influence of education on response. What do you conclude?

This is my DAG:

<img src="../../images/w7_img2.png" width="30%">

Here's the model we need, which includes education, age, and gender (female dummy variable):

In [26]:
with pm.Model() as model_12:
    # Data is defined outside of the model
    
    # Priors
    cutpoints = pm.Normal('cutpoints',
                          mu=0,
                          sd=1.5,
                          transform=pm.distributions.transforms.ordered,
                          shape=len(d.response.unique())-1,
                          testval=np.arange(len(d.response.unique())-1) - 2.5)
    bA = pm.Normal('bA', mu=0, sd=0.5)
    bC = pm.Normal('bC', mu=0, sd=0.5)
    bAge = pm.Normal('bAge', mu=0, sd=0.5)
    bE = pm.Normal('bE', mu=0, sd=0.5)
    bI = pm.Normal('bI', mu=0, sd=0.5)
    bM = pm.Normal('bM', mu=0, sd=0.5)
    bIA = pm.Normal('bIA', mu=0, sd=0.5)
    bIC = pm.Normal('bIC', mu=0, sd=0.5)
    edu_lvl_coefs = pm.Dirichlet('edu_lvl_coefs', a=np.ones(7))
    edu_lvl_aux = tt.concatenate([a, edu_lvl_coefs])
    
    # Regression
    BI = bI + bIA*action + bIC*contact
    phi = bA*action + bC*contact + bAge*age_std + BI*intention + bE*tt.sum(edu_lvl_aux[edu_new_total], axis=1) + bM*male
    response_hat = pm.OrderedLogistic('response_hat', phi, cutpoints, observed=response-1)

    # Prior sampling, trace definition and posterior sampling
    #prior = pm.sample_prior_predictive()
    posterior_2 = pm.sample()
    #posterior_pred_2 = pm.sample_posterior_predictive(posterior_2)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [edu_lvl_coefs, bIC, bIA, bM, bI, bE, bAge, bC, bA, cutpoints]
Sampling 4 chains, 0 divergences: 100%|██████████| 4000/4000 [10:12<00:00,  2.02draws/s] 
The acceptance probability does not match the target. It is 0.8863244726827754, but should be close to 0.8. Try to increase the number of tuning steps.
The estimated number of effective samples is smaller than 200 for some parameters.


In [27]:
az.summary(posterior_2, credible_interval=.89).round(2)

,mean,sd,mcse_mean,mcse_sd,hpd_5.5%,hpd_94.5%,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
bA,-0.48,0.05,0.00,0.00,-0.57,-0.40,1154.0,1145.0,1147.0,1695.0,1.00
bC,-0.34,0.07,0.00,0.00,-0.44,-0.23,1249.0,1249.0,1246.0,1430.0,1.00
bAge,-0.07,0.02,0.00,0.00,-0.11,-0.04,339.0,339.0,347.0,946.0,1.02
bE,0.02,0.21,0.01,0.01,-0.31,0.30,216.0,216.0,200.0,904.0,1.03
bI,-0.29,0.05,0.00,0.00,-0.38,-0.21,907.0,901.0,908.0,1285.0,1.00
bM,0.56,0.04,0.00,0.00,0.50,0.62,2410.0,2403.0,2398.0,1485.0,1.00
bIA,-0.44,0.08,0.00,0.00,-0.55,-0.32,1030.0,1030.0,1037.0,1548.0,1.00
bIC,-1.27,0.10,0.00,0.00,-1.42,-1.11,1110.0,1107.0,1115.0,1125.0,1.00
cutpoints[0],-2.36,0.16,0.01,0.01,-2.62,-2.12,275.0,275.0,275.0,781.0,1.02
cutpoints[1],-1.66,0.16,0.01,0.01,-1.92,-1.42,267.0,267.0,252.0,716.0,1.02


Age is still negative (and weak), while education is right near zero and straddles both sides. Gender seems to have accounted for all of the previous influenced assigned to education. It looks like female respondents gave lower average responses, indicating less approval.

It would be worth figuring out how gender is associated with education in this sample. It could be true for example that some education levels under-sampled men or women, and this leads to another kind of confound. Consider for example if older men are less likely to respond, so the sample becomes increasingly female with age. Then education level will also be increasingly female with age. Since the sample is
not a representative sample of the population, there are probably some biases of this sort.